In [ ]:
using Plots
gr()
include("core.jl")
include("var_types.jl")
include("algorithms.jl");

Core structs: 
```
struct Problem
    nobjs::Int64
    directions::Vector{Bool}
    nvars::Int64
    var_types::Vector{MOGA_Type}
end

mutable struct Solution
    problem::Problem
    x::Vector{Float64}
    evaluated::Bool
    objectives::Vector{Float64}
    crowding_distance::Float64
    rank::Int64
end
```

# Testing archive

In [ ]:
prob = Problem(2, [false, false], 5, [MOGA_Real(0,10) for i in 1:5]) # true means MINIMIZE, so we're maximizing here
sols = Vector{Solution}()
N = 10
for i=0:N
    r = i*(pi/2)/N
    objs = [cos(r), sin(r)]
    push!(sols, Solution(prob, [0.0], true, objs, 0.0, 0))
end

In [ ]:
archive = Archive(compare_pareto_dominance, Vector{Solution}())
insert_solutions!(archive, sols)
x = [s.objectives[1] for s in archive.solutions]
y = [s.objectives[2] for s in archive.solutions]
scatter(x, y, size=(300,300), aspect_ratio=:equal, legend=false)

In [ ]:
s1 = Solution(prob, [0.0], true, [.5, .5], 0.0, 0)
s2 = Solution(prob, [0.0], true, [.9, .9], 0.0, 0)

In [ ]:
insert_solutions!(archive, [s1])
x = [s.objectives[1] for s in archive.solutions]
y = [s.objectives[2] for s in archive.solutions]
scatter(x, y, size=(300,300), aspect_ratio=:equal, legend=false)

In [ ]:
insert_solutions!(archive, [s2])
x = [s.objectives[1] for s in archive.solutions]
y = [s.objectives[2] for s in archive.solutions]
scatter(x, y, size=(300,300), aspect_ratio=:equal, legend=false)

# Testing non-dominated sort

In [ ]:
prob = Problem(2, [false, false], 5, [MOGA_Real(0,10) for i in 1:5]) # true means MINIMIZE, so we're maximizing here
sols = Vector{Solution}()
N = 10
for i=0:N
    r = i*(pi/2)/N
    objs = [cos(r), sin(r)]
    push!(sols, Solution(prob, [0.0], true, objs, 0.0, 0))
    push!(sols, Solution(prob, [0.0], true, .9*objs, 0.0, 0))
    push!(sols, Solution(prob, [0.0], true, .8*objs, 0.0, 0))
end
nondominated_sort(sols)

In [ ]:
x = [s.objectives[1] for s in sols]
y = [s.objectives[2] for s in sols]
r = [s.rank for s in sols]
cdist = [s.crowding_distance for s in sols]
cdist[isinf.(cdist)] = 0
p1 = scatter(x, y, zcolor=r, aspect_ratio=:equal, legend=false, title="rank")
p2 = scatter(x, y, zcolor=cdist, aspect_ratio=:equal, legend=false, title="crowding distance")
plot(p1, p2, layout=(1, 2))